<a href="https://colab.research.google.com/github/perv0vser/DA_test_projects/blob/main/%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D0%B7%D0%B0%D0%BF%D1%80%D0%BE%D1%81_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B8%D0%B7_%D0%91%D0%94_SQL_%D0%BF%D1%80%D0%B5%D0%B4%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_Pandas_%D0%B4%D0%B0%D1%88%D0%B1%D0%BE%D1%80%D0%B4_DataLens_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача 1: Выгрузить из базы данных с использованием SQL-запроса информацию по следующим пользователям:

- Ученики с годовых курсов ЕГЭ и ОГЭ

Необходимые поля в итоговой выгрузке:

- ID курса
- Название курса
- Предмет
- Тип предмета
- Тип курса
- Дата старта курса
- ID ученика
- Фамилия ученика
- Город ученика
- Ученик не отчислен с курса
- Дата открытия курса ученику
- Сколько полных месяцев курса открыто у ученика
- Число сданных ДЗ ученика на курсе

Файлы, необходимые для решения задачи:

- db_schema.jpg – схема БД
- script_db.py – python-скрипт для имитации запроса к БД

Результат, который нужно прислать по задаче:

- SQL-скрипт в формате файла .sql. В коде скрипта оставляйте комментарии с пояснениями своих действий

In [ ]:
# Данный python-скрипт имитирует запрос к БД
# Напишите ваш SQL-запрос в query и запустите данный python-скрипт для получения результата
# Перед запуском скрипта установите библиотеку duckdb

# Установка библиотеки duckdb
# pip install duckdb duckdb-engine

In [ ]:
# Импорт библиотек
import pandas as pd
import duckdb

In [ ]:
# Задание таблиц БД
users = pd.read_csv('/content/users.csv')
course_users = pd.read_csv('/content/course_users.csv')
courses = pd.read_csv('/content/courses.csv')
course_types = pd.read_csv('/content/course_types.csv')
lessons = pd.read_csv('/content/lessons.csv')
subjects = pd.read_csv('/content/subjects.csv')
cities = pd.read_csv('/content/cities.csv')
homework_done = pd.read_csv('/content/homework_done.csv')
homework = pd.read_csv('/content/homework.csv')
homework_lessons = pd.read_csv('/content/homework_lessons.csv')
user_roles = pd.read_csv('/content/user_roles.csv')

Ссылка на файл со скриптом: https://disk.yandex.ru/d/UQdgTwlMS5v3kg

In [ ]:
query = """
SELECT c.id AS course_id, -- Здесь и далее при задании alias для таблиц использую пробел, для столбцов (полей) оператор AS
       c.name AS course_name,
       s.name AS subject_name,
       s.project AS subject_type,
       ct.name AS course_type,
       CAST(c.starts_at AS date) AS start_date,
       u.id AS user_id,
       u.last_name AS user_name,
       ci.name AS user_city,
       cu.active AS user_status,
       CAST(cu.created_at AS date) AS user_joined,
       /*Целочисленное деление на 30 количества дней между датами показалось не совсем корректным решением,
       так как месяцы идут с разным кол-вом дней, решил применить такой код, но наиболее корректное решение можно оформить,
       если пообщаться с заказчиком таблицы и дашборда и лучше понять, по какому принципу вносятся обновления в таблицу course_users*/
       EXTRACT(MONTH FROM age(CAST(cu.updated_at AS date), CAST(cu.created_at AS date))) AS full_months,
       hd.count AS homeworks_done

FROM courses c
JOIN subjects s ON c.subject_id = s.id -- Здесь и далее использую INNER JOIN, чтобы оставались только пересекающиеся данные
JOIN (
           SELECT *
           FROM course_types
           WHERE name = 'Годовой'
) ct ON c.course_type_id = ct.id
JOIN course_users cu ON c.id = cu.course_id
JOIN users u ON cu.user_id = u.id
JOIN (
           SELECT *
           FROM user_roles
           WHERE name = 'student' -- Фильтровать ненужные данные перед объединением таблиц кажется логичнее
) ur ON u.user_role_id = ur.id
JOIN cities ci ON u.city_id = ci.id
JOIN (
          SELECT user_id,
                 COUNT(homework_id) AS count -- Агрегируем перед объединением, тем более user_id является ключем, а д/з у каждого может быть несколько
          FROM homework_done
          GROUP BY user_id
) hd ON u.id = hd.user_id
"""

In [ ]:
# Выполнение SQL-запроса
df_result = duckdb.query(query).to_df()

In [ ]:
df_result

,course_id,course_name,subject_name,subject_type,course_type,start_date,user_id,user_name,user_city,user_status,user_joined,full_months,homeworks_done
0,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10002,Коваленко,Москва,0,2024-07-25,3,5
1,25,Годовой 2к25 стандарт,Биология,ЕГЭ,Годовой,2024-09-17,10005,Гусев,Белгород,1,2024-07-26,2,2
2,23,Годовой 2К25 стандарт с Машей,Математика,ЕГЭ,Годовой,2024-09-20,10011,Ищенко,Анива,1,2024-09-10,1,11
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10020,Бархатова,Москва,1,2024-07-26,2,13
4,17,Годовой 2к25 стандарт,Английский язык,ЕГЭ,Годовой,2024-09-20,10023,Накагава,Холмск,1,2024-07-26,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,25942,Такой,Краснодар,1,2024-09-17,1,21
5105,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,25975,Aix,Краснодар,1,2024-09-17,1,31
5106,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,28022,Зотова,Челябинск,1,2024-09-23,0,47
5107,36,Годовой курс 2к25,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-30,28327,Юшкова,Серов,1,2024-09-24,0,7


# Задача 2: С использованием python-скриптов обработать датасет, подготовленный в Задаче 1

1. Проверить датасет на: дубликаты, пропуски, типы данных, аномальные значения. Произвести
предобработку датасета, если это необходимо
2. Поделить всех учеников на волны
Критерии волн:
- 0 волна – присоединились на курс до даты старта курса включительно
- 1 волна - присоединились на курс в течение 1 недели после даты старта курса включительно
- 2 волна - присоединились на курс в течение от 1 до 2 недель включительно после даты старта курса
- 3 волна - присоединились на курс в течение от 2 до 3 недель включительно после даты старта курса
- 4 волна - присоединились на курс в течение от 3 до 4 недель включительно после даты старта курса
- 5 волна - присоединились на курс от 4 недель после даты старта курса

In [ ]:
!pip install transliterate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.6 MB/s eta 0:00:00


In [ ]:
import re
from transliterate import translit
#from transliterate.decorators import transliterate_function

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
df_result

,course_id,course_name,subject_name,subject_type,course_type,start_date,user_id,user_name,user_city,user_status,user_joined,full_months,homeworks_done
0,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10002,Коваленко,Москва,0,2024-07-25,3,5
1,25,Годовой 2к25 стандарт,Биология,ЕГЭ,Годовой,2024-09-17,10005,Гусев,Белгород,1,2024-07-26,2,2
2,23,Годовой 2К25 стандарт с Машей,Математика,ЕГЭ,Годовой,2024-09-20,10011,Ищенко,Анива,1,2024-09-10,1,11
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10020,Бархатова,Москва,1,2024-07-26,2,13
4,17,Годовой 2к25 стандарт,Английский язык,ЕГЭ,Годовой,2024-09-20,10023,Накагава,Холмск,1,2024-07-26,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,25942,Такой,Краснодар,1,2024-09-17,1,21
5105,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,25975,Aix,Краснодар,1,2024-09-17,1,31
5106,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,28022,Зотова,Челябинск,1,2024-09-23,0,47
5107,36,Годовой курс 2к25,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-30,28327,Юшкова,Серов,1,2024-09-24,0,7


## EDA

In [ ]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5109 entries, 0 to 5108
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   course_id       5109 non-null   int64         
 1   course_name     5109 non-null   object        
 2   subject_name    5109 non-null   object        
 3   subject_type    5109 non-null   object        
 4   course_type     5109 non-null   object        
 5   start_date      5109 non-null   datetime64[us]
 6   user_id         5109 non-null   int64         
 7   user_name       5109 non-null   object        
 8   user_city       5109 non-null   object        
 9   user_status     5109 non-null   int64         
 10  user_joined     5109 non-null   datetime64[us]
 11  full_months     5109 non-null   int64         
 12  homeworks_done  5109 non-null   int64         
dtypes: datetime64[us](2), int64(5), object(6)
memory usage: 519.0+ KB


In [ ]:
df_result.duplicated().sum()

0

In [ ]:
def check_anomalies(df):
    '''Выводит списки уникальных значений столбцов датафрейма'''
    for col in df.columns.tolist():
        print(f'Уникальные значения в столбце {col}')
        print(df[col].unique())
        print('*'*100)

In [ ]:
check_anomalies(df_result)

Уникальные значения в столбце course_id
[19 25 23 16 17 24 42 22 37 18 38 39 36 41 21 20 81 62 80 57 59 74 79 56
 73]
****************************************************************************************************
Уникальные значения в столбце course_name
['Годовой 2к25 стандарт' 'Годовой 2К25 стандарт с Машей'
 'В погоне за пятеркой 2К25 стандарт' 'Годовой курс 2к25'
 'Годовой курс 2к25 комбо (русский + обществознание + математика)'
 'Годовой 2К25 стандарт с Катей' 'Годовой 2к25 минимал'
 'Годовой 2к25 платинум'
 'Годовой курс 2к25 комбо (биология + обществознание + математика)'
 'Годовой курс 2к25 комбо (биология + русский + математика)']
****************************************************************************************************
Уникальные значения в столбце subject_name
['Русский' 'Биология' 'Математика' 'Обществознание' 'Английский язык'
 'Математика (база)' 'Биология ОГЭ' 'Литература' 'Русский ОГЭ' 'История'
 'Математика ОГЭ' 'Обществознание ОГЭ' 'Химия' 'Физика']
**

In [ ]:
sorted(df_result.user_city.unique())[:100]

['Абакан',
 'Агидель',
 'Агрыз',
 'Адыгейск',
 'Азнакаево',
 'Азов',
 'Аксай',
 'Алдан',
 'Александровск',
 'Алексеевка',
 'Алексин',
 'Алушта',
 'Альметьевск',
 'Анапа',
 'Ангарск',
 'Андреаполь',
 'Анжеро-Судженск',
 'Анива',
 'Апатиты',
 'Апшеронск',
 'Арзамас',
 'Аркадак',
 'Армавир',
 'Арск',
 'Артём',
 'Архангельск',
 'Асбест',
 'Астрахань',
 'Ахтубинск',
 'Ачинск',
 'Аша',
 'Бабаево',
 'Бавлы',
 'Баксан',
 'Балабаново',
 'Балаково',
 'Балахна',
 'Балашиха',
 'Балтийск',
 'Барабинск',
 'Барнаул',
 'Белгород',
 'Белебей',
 'Белинский',
 'Белово',
 'Белогорск',
 'Белокуриха',
 'Белорецк',
 'Белореченск',
 'Белоярский',
 'Бердск',
 'Бердянск\r',
 'Берёзовский',
 'Беслан',
 'Бийск',
 'Биробиджан',
 'Бирск',
 'Бирюч',
 'Благовещенск',
 'Богородицк',
 'Богучар',
 'Болгар',
 'Бор',
 'Борисоглебск',
 'Боровичи',
 'Братск',
 'Бронницы',
 'Брянск',
 'Бугульма',
 'Бугуруслан',
 'Будённовск',
 'Буй',
 'Валдай',
 'Великие Луки',
 'Великий Новгород',
 'Великий Устюг',
 'Вельск',
 'Верхний Уфал

In [ ]:
sorted(df_result.user_name.unique())[:500]

['A',
 'Abdulaeva',
 'Abovyan',
 'Abramyan',
 'Ailasova',
 'Airian',
 'Aix',
 'Akhmedova',
 'Aleks',
 'Aleksandrovna',
 'Aleshina',
 'Alexandrova',
 'Almazovna',
 'Alone',
 'Andre',
 'Andreeva',
 'Andreevna',
 'Anisimova',
 'Ankha',
 'Artemova',
 'Arustamyan',
 'Astrova',
 'Atokurieta',
 'Aymi',
 'Azumane',
 'Bahng',
 'Bakuta',
 'Bang',
 'Bars',
 'Barsova',
 'Blades',
 'Bochkova',
 'Bogdanova',
 'Bro',
 'Bugrov',
 'Bulgakova',
 'Bzaubakova',
 'Cartier',
 'Chakhmakhchyan',
 'Cheese',
 'Chernova',
 'Chicareva',
 'Christian',
 'Chukhno',
 'Co',
 'Crimson',
 'Danilina',
 'Dasha',
 'Denisovna',
 'Diliara',
 'Dinosaurs',
 'Doronenko',
 'Doroshenko',
 'Egoraeva',
 'Egorikov',
 'Eliss',
 'Erohov',
 'Evgraf',
 'Fakhretdinova',
 'Family',
 'Fatykhova',
 'Fazilova',
 'Filippova',
 'Finaev',
 'Fist',
 'Fly',
 'Fodeeva',
 'Fox',
 'Frolova',
 'Frr',
 'G',
 'Gabuev',
 'Gachechiladze',
 'Gavre',
 'Gaynutdinov',
 'Getmanova',
 'Glyatsevich',
 'God',
 'Gold',
 'Goldina',
 'Golikova',
 'Grey',
 'Grigorya

In [ ]:
sorted(df_result.user_id.unique())[:10], sorted(df_result.user_id.unique())[-10:]

([10001, 10002, 10005, 10006, 10009, 10010, 10011, 10012, 10014, 10017],
 [33986, 33989, 33990, 33991, 33993, 33994, 33996, 33997, 33998, 33999])

**Комментарий:** Пропусков нет, дубликатов тоже. Типы данных адекватные. Есть аномалии в наименованиях городов и фамилиях. В первом случае - это один паттерн в виде сочетания символов '\r' в русскоязычном тексте. Можно было бы отделаться заменой, но я применил регулярку, чтобы убрать вообще все лишние символы, чтобы показать, что я влядею инструментом и для масштабируемости кода. Однако, если в новых данных возникнет транслит, как в фамилиях, то нужно будет пересмотреть решение проблемы, иначе названия на транслите просто пропадут. Транслит в фамилиях перевел с помощью специальной библиотеки и небольших "костылей" к ней. Решения и выводы см. ниже.

### Корректировка тектовых значений

In [ ]:
def clear_city(text):
  '''Принимает на вход строку, возвращает строку очищенную от нерусскоязычных символов'''
  return re.sub(r'[^А-Яа-яЁё -]', '', text)

In [ ]:
df_result.user_city = df_result.user_city.apply(clear_city)

In [ ]:
sorted(df_result.user_city.unique().tolist())

['Абакан',
 'Агидель',
 'Агрыз',
 'Адыгейск',
 'Азнакаево',
 'Азов',
 'Аксай',
 'Алдан',
 'Александровск',
 'Алексеевка',
 'Алексин',
 'Алушта',
 'Альметьевск',
 'Анапа',
 'Ангарск',
 'Андреаполь',
 'Анжеро-Судженск',
 'Анива',
 'Апатиты',
 'Апшеронск',
 'Арзамас',
 'Аркадак',
 'Армавир',
 'Арск',
 'Артём',
 'Архангельск',
 'Асбест',
 'Астрахань',
 'Ахтубинск',
 'Ачинск',
 'Аша',
 'Бабаево',
 'Бавлы',
 'Баксан',
 'Балабаново',
 'Балаково',
 'Балахна',
 'Балашиха',
 'Балтийск',
 'Барабинск',
 'Барнаул',
 'Белгород',
 'Белебей',
 'Белинский',
 'Белово',
 'Белогорск',
 'Белокуриха',
 'Белорецк',
 'Белореченск',
 'Белоярский',
 'Бердск',
 'Бердянск',
 'Берёзовский',
 'Беслан',
 'Бийск',
 'Биробиджан',
 'Бирск',
 'Бирюч',
 'Благовещенск',
 'Богородицк',
 'Богучар',
 'Болгар',
 'Бор',
 'Борисоглебск',
 'Боровичи',
 'Братск',
 'Бронницы',
 'Брянск',
 'Бугульма',
 'Бугуруслан',
 'Будённовск',
 'Буй',
 'Валдай',
 'Великие Луки',
 'Великий Новгород',
 'Великий Устюг',
 'Вельск',
 'Верхний Уфалей

In [ ]:
def translit_name(name):
  '''Принимает на вход строку, содержащую транслит, возвращает строку в русскоязычных символах'''
  ru_name = translit(name, 'ru')
  ru_name = ru_name.replace('W', 'В').replace('w', 'в').replace('X', 'Кс')
  return ru_name

In [ ]:
df_result.user_name = df_result.user_name.apply(translit_name)

In [ ]:
sorted(df_result.user_name.unique().tolist())

['А',
 'Абастов',
 'Абб',
 'Абдувелиева',
 'Абдулаева',
 'Абдулина',
 'Абдуллаев',
 'Абдуллаева',
 'Абдуллина',
 'Абдухамидова',
 'Абзалова',
 'Абибуллаев',
 'Аблаева',
 'Абовыан',
 'Абрамова',
 'Абрамушкина',
 'Абрамыан',
 'Абсаидов',
 'Абубакирова',
 'Абшцы',
 'Авдеенко',
 'Аверина',
 'Аветисян',
 'Авксентьев',
 'Аврам',
 'Авраменко',
 'Аврора',
 'Австрина',
 'Агаева',
 'Агальцова',
 'Агатова',
 'Агеев',
 'Агеева',
 'Аглиева',
 'Аглиуллина',
 'Адикаева',
 'Азизбаева',
 'Азизова',
 'Азова',
 'Азумане',
 'Аиx',
 'Аиласова',
 'Аириан',
 'Аистова',
 'Айзенов',
 'Акар',
 'Акимова',
 'Акобян',
 'Аксенов',
 'Аксёнова',
 'Акулов',
 'Акулова',
 'Акхмедова',
 'Акылбаева',
 'Алеxандрова',
 'Алекс',
 'Александрова',
 'Александровна',
 'Алексеев',
 'Алексеева',
 'Алексеевна',
 'Аленькова',
 'Алехина',
 'Алешина',
 'Алибаева',
 'Алиева',
 'Алимгереева',
 'Алимов',
 'Алимова',
 'Алиса',
 'Алисенова',
 'Аллабердиева',
 'Аллахвердова',
 'Алмаева',
 'Алмазовна',
 'Алоне',
 'Алпеев',
 'Алтунян',
 'Алты

**Комментарий:** Что касается городов, то проделанная предобработка полностью решила все аномалии в данных. Касательно фамилий - я только решил проблему транслитерации, однако остаются аномальные значения фамилий, например, состоящие из одной буквы - это уже нужно обсуждать с заказчиком, где искать корректные фамилии по имеющимся ID, так что пока оставляю в таком виде. Еще у библиотеки transliterate проблемы с переводом символов "w" и "x", возможно еще каких-то символов или сочетаний, которые нам не встретились. С коде это лечится дополнительными заменами, но я прописал замены только тех символов, что встретились в нашем датасете. Если делать код применимым для новых данных, то следует пересмотреть словарь символов под замену или подобрать более корректно работающую библиотеку для "лечкения" транслита.

 ##  Добавление волн

**Комментарий:** Сначала добавляем новый признак (разницу между датой присоединения студента и датой начала курса): если разница отрицательная, то студент записался до старта курса, если равна 0, то он записался в день старта курса. Все положительные значения соответствуют количеству дней после старта курса (временному лагу записи на курс в днях).

In [ ]:
df_result['join_lag'] = (df_result['user_joined'] - df_result['start_date']).dt.days

In [ ]:
df_result

,course_id,course_name,subject_name,subject_type,course_type,start_date,user_id,user_name,user_city,user_status,user_joined,full_months,homeworks_done,join_lag
0,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10002,Коваленко,Москва,0,2024-07-25,3,5,-58
1,25,Годовой 2к25 стандарт,Биология,ЕГЭ,Годовой,2024-09-17,10005,Гусев,Белгород,1,2024-07-26,2,2,-53
2,23,Годовой 2К25 стандарт с Машей,Математика,ЕГЭ,Годовой,2024-09-20,10011,Ищенко,Анива,1,2024-09-10,1,11,-10
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10020,Бархатова,Москва,1,2024-07-26,2,13,-52
4,17,Годовой 2к25 стандарт,Английский язык,ЕГЭ,Годовой,2024-09-20,10023,Накагава,Холмск,1,2024-07-26,2,10,-56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,25942,Такой,Краснодар,1,2024-09-17,1,21,-4
5105,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,25975,Аиx,Краснодар,1,2024-09-17,1,31,-8
5106,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,28022,Зотова,Челябинск,1,2024-09-23,0,47,-2
5107,36,Годовой курс 2к25,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-30,28327,Юшкова,Серов,1,2024-09-24,0,7,-6


In [ ]:
def categorize_waves(lag):
    '''Категоризирует учеников по соответствующим волнам в зависимости от значения временного лага записи на курс'''
    try:
        if lag <= 0:
            return '0'
        elif 0 < lag <= 7:
            return '1'
        elif 7 < lag <= 14:
            return '2'
        elif 14 < lag <= 21:
            return '3'
        elif 21 < lag <= 28:
            return '4'
        elif lag > 28:
            return '5'
    except:
        pass

In [ ]:
df_result['wave'] = df_result['join_lag'].apply(categorize_waves)

In [ ]:
df_result

,course_id,course_name,subject_name,subject_type,course_type,start_date,user_id,user_name,user_city,user_status,user_joined,full_months,homeworks_done,join_lag,wave
0,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10002,Коваленко,Москва,0,2024-07-25,3,5,-58,0
1,25,Годовой 2к25 стандарт,Биология,ЕГЭ,Годовой,2024-09-17,10005,Гусев,Белгород,1,2024-07-26,2,2,-53,0
2,23,Годовой 2К25 стандарт с Машей,Математика,ЕГЭ,Годовой,2024-09-20,10011,Ищенко,Анива,1,2024-09-10,1,11,-10,0
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10020,Бархатова,Москва,1,2024-07-26,2,13,-52,0
4,17,Годовой 2к25 стандарт,Английский язык,ЕГЭ,Годовой,2024-09-20,10023,Накагава,Холмск,1,2024-07-26,2,10,-56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,25942,Такой,Краснодар,1,2024-09-17,1,21,-4,0
5105,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,25975,Аиx,Краснодар,1,2024-09-17,1,31,-8,0
5106,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,28022,Зотова,Челябинск,1,2024-09-23,0,47,-2,0
5107,36,Годовой курс 2к25,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-30,28327,Юшкова,Серов,1,2024-09-24,0,7,-6,0


In [ ]:
df_result['wave'].value_counts(normalize=True)*100

,proportion
wave,
0,37.48
3,16.87
2,16.70
1,14.33
4,14.13
5,0.49


In [ ]:
df_result.to_csv('df_result.csv', index=False)

# Задача 3:

Построить дашборд в Yandex DataLens с использованием датасета, подготовленного в Задаче 2.
Цели дашборда:
1. Отслеживать какие курсы хуже всего /лучше всего продлеваются учениками
2. Отбирать учеников, которые не продлили курс, по дополнительным заданным критериям, чтобы
связываться с данными учениками для повышения процента продлений

Для реализации данных целей на дашборде должна быть возможность:

1. Сравнивать курсы по двум метрикам: числу и проценту продлений заданного месяца.

Пример: на заданном курсе всего 1000 учеников, из них продлили 2-й месяц курса 800 учеников. Тогда на данном курсе: число продлений 2-го месяца равно 800, процент продлений 2-го месяца равен 80%

2. Получать полную информацию по ученикам (курс ученика, волна ученика на курсе, число сданных ДЗ ученика на курсе, город ученика и т.д.), которые подходят под данное условие:

- Не отчислены с курса

Также должна быть возможность среди данных учеников отобрать учеников:

- С заданных курсов
- С заданных волн
- С заданных городов
- Которые не продлили заданный месяц курса
- У которых число сданных ДЗ на курсе меньше заданного числа


**Ссылка на дашборд с публичным доступом:** https://datalens.yandex/wcw28pus37puj

Касаемо кода и логики, то только в одном чарте была использована формула для расчета доли продливших курс на заданный месяц от всех учеников курса: 100 * COUNT([Действующий ученик]) / SUM(COUNT([Действующий ученик]) WITHIN [Название курса],[Старт курса]). Логика тут довольно простая, не уверен, что нужно ее объяснять. Сам дашборд получился не самым "нарядным", но функционален и решает поставленные в ТЗ задачи.